<a href="https://colab.research.google.com/github/wouterhuls/FlavourPhysicsBND2023/blob/main/mixingfrequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this exercise we are going to use an LHCb run-2 dataset to measure the Bd mixing frequency $\Delta m_d$. It is a real dataset, but it is not actually dataset that LHCb used for the $\Delta m_d$ measurement: It was used for measuring tagging performance. Yet, the result will be reaonable competitive with the world average.

The learning goals of this exercise are:
* fitting with zfit, a python based modeling package based on Tensorflow. For more instructions, visit the zfit documentation.
* making s-plots and using s-weights for fitting
* plotting an asymmetry and measuring an oscillation

The analysis will consist of two steps. In the first step you will learn about zfit and s-weights. In the second step you will measure the mixing frequency.

# The physics

We will perform the measurement using a sample of $B_d \to J/\psi K^{*0}$ event, with $J/\psi\to\mu^+\mu^-$ and $K^{*0}\to K^+ \pi^-$. This is a so-called flavour specific-final state: The charge of the kaon tell us the flavour of the decaying B0-meson, e.g. whether it was decaying as a $B^0$ or $\bar{B}^0$.

To measure the time-dependent oscillations we need two ingredients, namely:
* the decay time
* the flavour of the B at production
You have learned in the lectures that the the rate for a B produced as $B^0$ to decay as $B^0$ is given by

$ N( B^0 \to B^0 ) = \frac{e^{-t/\tau}}{2} ( 1 + \cos( \Delta m t) ) $

$ N( B^0 \to \bar{B}^0 ) = \frac{e^{-t/\tau}}{2} ( 1 - \cos( \Delta m t) ) $

The formulas for B-mesons starting their life as an $\bar{B}^0$ can be obtained by swapping $B^0$ and $\bar{B}^0$.

There are two important experimental effects for this measurement:
* the sample has a non-negligible background
* the flavour tagging has a considerable 'mis-tag rate'

In the following we will have to deal with these two effects.




In [1]:

!conda install uproot --channel conda-forge

3.10.12
/bin/bash: line 1: conda: command not found


# Exercise 1

Look up in the PDG (google `pdg live`) the quark content of these mesons: $B^0$, $J/\psi$, $K^{*0}$. Draw the Feynman diagram for the decay $B^0 \to J/\psi \bar{K}^{*0}$ (on a piece of paper, or on your tablet.)


# Prerequites

Install the `zfit` package.

In [ ]:
# @ Prerequisites
import platform
print(platform.python_version())

#!conda install uproot --channel conda-forge
#!conda install zfit --channel conda-forge
#!conda install hepstats --channel conda-forge
#!conda install mlphep --channel conda-forge
# In google colab, use pip rather than conda
!pip install zfit
!pip install hepstats
!pip install mplhep
!pip install uproot


# Exercise 2

Attached to this workbook you will find a 'ROOT' file. The file contains a TTree (named `tree`) with a number of fields (called `branches` in ROOT language). For the rest of the exercise, the relevant fields are:
* `mass`: the B candidate invariant mass in MeV
* `decaytime`: the B candidate decaytime in ns
* `q`: the charge of the B candidate reconstructed by the flavour tagging algorithm
* `eta`: the mistagrate assigned by the flavour tagging algorithm
* `pid`: the PDG value that the LHCb software assigned to the decaying B: This can be either 512 (for $B^0$) or -512 (for anti-$B^0$), depending on whether the kaon was $K^+$ or $K^-$.

Load the dataset with your favourite tool and draw the reconstructed invariant mass. If you plot it on a log scale, you will find one peak on a falling exponential background.

In [ ]:
! pip install uproot
import uproot

In [ ]:
#@title Example solution
# This is a partial solution to the exercise using uproot. You can also use pyroot if you prefer.
url = 'http://www.nikhef.nl/~wouterh/tmp/kstarntuple_for_BND.root'

import uproot
events = uproot.open(url + ":tree")
mass = events["mass"].array()

import matplotlib.pyplot as plt
plt.hist(mass, bins=200)
plt.show()

# Hint: you will see a lot more if you plot on a log scale and with more bins!

# Exercise 3

Draw also the B candidate 'decaytime'. The units are in nanoseconds. Compute the average decaytime and its statistical error. How does the answer compare to the average $B^0$ lifetime in the PDG? Give two reasons why the two are different.


# Exercise 4

We will now perform a fit to the invariant mass distribution to extract the number of $B^0$ events. Because it may take you too much time to figure this out yourself, we have written most of the code for you.

If you look at the final fit result superimposed on the data set, it looks pretty bad. One reason is the 'signal mass model': it is not very well described by a Gaussian.

In [ ]:
import zfit
import numpy as np

# Specify the mass range. To simplify the fit, we first limit the mass range to the region just around the B0 mass peak.
massmin = 5150
massmax = 5350

# temporary hack, to make sure we can rerun this cell as often as we like.
from collections import OrderedDict
zfit.core.parameter.ZfitParameterMixin._existing_params = OrderedDict()

# use uproot to get the subset of events in this mass range in numpy format
npevents = events.arrays( library='np',expressions=['mass','decaytime','masserr'] )
mass = npevents[ 'mass' ]
masserr = npevents[ 'masserr' ]
decaytime = npevents['decaytime']

# use a little bit complicated logic to get access to the events in the narrow
# mass range in such a way that we can still identify the events in the
# original list. (we need that later)
mask = np.logical_and(mass>massmin,mass<massmax,masserr<10)
indices = np.where(mask)

# create a zfit data set from the numpy array. when constructing a zfit dataset
# from a numpy array we need to tell how we 'name' the columns
massobs = zfit.Space("mass",(massmin,massmax))
zdata = zfit.Data.from_numpy( array = mass[indices], obs = massobs )

# create a zfit pdf for the B0 signal
mu_B0 = zfit.Parameter("mu_B0", 5279, 5250, 5300)
sigma_B0 = zfit.Parameter("sigma_B0", 10, 0, 30)
masspdf_B0 = zfit.pdf.Gauss(mu=mu_B0, sigma=sigma_B0, obs=massobs)

# create a zfit pdf for the exponential background
lambd = zfit.Parameter("lambda", -0.001, -1,+1)
masspdf_bkg = zfit.pdf.Exponential(lambd, obs=massobs)

# create an extended PDF from the sum of these
nev = len( mass )
yield_B0  = zfit.Parameter("yield_B0", 0.9*nev, -0.1*nev, 1.1*nev)
yield_bkg = zfit.Parameter("yield_bkg", 0.1*nev, -0.1*nev, 1.1*nev)
extmasspdf_B0  = masspdf_B0.create_extended(yield_ = yield_B0)
extmasspdf_bkg = masspdf_bkg.create_extended(yield_ = yield_bkg)
pdf_total  = zfit.pdf.SumPDF([extmasspdf_B0, extmasspdf_bkg], name="totPDF")

# create a loss function. this is what we will 'minimize'
nll_data = zfit.loss.ExtendedUnbinnedNLL(model=pdf_total, data=zdata)
# create the minimizer. This one uses minuit, but there are various alternatives.
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(nll_data)
result.hesse()
print(result)

# draw the result
n_bins = 200
npdata = zdata['mass'].numpy()
plot_scaling = len(npdata) / n_bins * massobs.area()
x = np.linspace(massmin,massmax, 1000)
y = pdf_total.pdf(x).numpy()
fig, axes = plt.subplots(2)
axes[1].set_yscale("log")
for i in range(2):
  axis = axes[i]
  color = 'black'
  axis.hist(npdata, color=color, bins=n_bins, histtype="stepfilled", alpha=0.1)
  axis.hist(npdata, color=color, bins=n_bins, histtype="step")
  axis.plot(x, y * plot_scaling, label="Sum - Model", linewidth=2)
  axis.set_xlabel("mass [MeV]")
plt.show()


In [ ]:
# repeat the fit but with a better mass model
zfit.core.parameter.ZfitParameterMixin._existing_params = OrderedDict()

aL = zfit.Parameter("aL_B0",  1.4, 0.1, 5,floating=True)
aR = zfit.Parameter("aR_B0",  1.4, 0.1, 5,floating=True)
aR = aL
nL = zfit.Parameter("nL_B0", 6, 1., 10, floating=True)
nR = zfit.Parameter("nR_B0", 10, 1., 20,floating=True)

masspdf_B0 = zfit.pdf.DoubleCB(obs=massobs, mu=mu_B0, sigma=sigma_B0, alphal=aL, nl=nL, alphar=aR, nr=nR)

extmasspdf_B0  = masspdf_B0.create_extended(yield_ = yield_B0)
extmasspdf_bkg = masspdf_bkg.create_extended(yield_ = yield_bkg)
pdf_total  = zfit.pdf.SumPDF([extmasspdf_B0, extmasspdf_bkg], name="totPDF")

nll_data = zfit.loss.ExtendedUnbinnedNLL(model=pdf_total, data=zdata)
# create the minimizer. This one uses minuit, but there are various alternatives.
result = minimizer.minimize(nll_data)
result.hesse()
print(result)

# draw the result
n_bins = 200
npdata = zdata['mass'].numpy()
plot_scaling = len(npdata) / n_bins * massobs.area()
x = np.linspace(massmin,massmax, 1000)
y = pdf_total.pdf(x).numpy()
fig, axes = plt.subplots(2)
axes[1].set_yscale("log")
for i in range(2):
  axis = axes[i]
  color = 'black'
  axis.hist(npdata, color=color, bins=n_bins, histtype="stepfilled", alpha=0.1)
  axis.hist(npdata, color=color, bins=n_bins, histtype="step")
  axis.plot(x, y * plot_scaling, label="Sum - Model", linewidth=2)
  axis.set_xlabel("mass [MeV]")
plt.show()


In [ ]:
# @ compute s-weights and make a decaytime plot

# we can get rid of the masks if we only work with the selected events. that actually makes a bit more sense.
from hepstats.splot import compute_sweights
sweights_all = compute_sweights(pdf_total, npdata)
sweights_B0 = np.zeros_like( mass, dtype=np.float64 )
np.place(sweights_B0, mask, sweights_all[yield_B0] )


In [ ]:
# plot the s-weighted decay time distribution
import matplotlib.pyplot as plt
decaytime = npevents['decaytime']
q = events["q"].array()
eta = events["eta"].array()
plt.hist(decaytime, bins=200, weights = sweights_B0 * (pid*q<0)* (1-2*eta))
plt.hist(decaytime, bins=200, weights = sweights_B0 * (pid*q>0)* (1-2*eta))
plt.show()

# suggested binning for asymmetry plot
#tbins = np.concatenate((np.arange(0.0000,5.0,step=0.5),[5.6,6.2,7.0,7.8,8.7,9.7,10.8,12.0,13.4,15.]))
tbins = np.array([0.002,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.2,6.2,7.5,8.8,10.2,11.5,15.])
tbins = tbins/1000.

# choose 20 bins with equal number of events
#tbins = np.quantile(decaytime, np.linspace(start=0.0,stop=1.0,num=21)[1:])
#tbins[-1] = 0.015
#tbins[0]  = 0.0002

# I'm not sure how to do this properly
qD = q*(1-2*eta)*-1
wqDsum, bin_edges  = np.histogram(decaytime,bins=tbins,weights=sweights_B0*qD)
wqD2sum, bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0*qD*qD)
w2qD2sum, bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0*sweights_B0*qD*qD)
asymmetry    = wqDsum / wqD2sum
asymmetryerr = np.sqrt(w2qD2sum) / wqD2sum

# compute in every bin the average decay time
wtsum, bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0*decaytime)
wsum,  bin_edges = np.histogram(decaytime,bins=tbins,weights=sweights_B0)
avtime = wtsum / wsum

# now draw points with both vertical and horizontal errors
xerrors = [avtime-bin_edges[:-1],bin_edges[1:]-avtime]
plt.errorbar(x=avtime, y=asymmetry, xerr=xerrors, yerr=asymmetryerr,fmt='o')
plt.show()



# compute s-weights corresponding to each of the yields

In [ ]:
# define a model and fit to s-weighted data

# for now, just measure the amplitude of the sin-wave
# this only works because C=0
deltaM = 507. # mixing frequency in [2pi/ns]
qDsin = qD * np.sin(decaytime*deltaM)
wqDsinsum = np.sum( sweights_B0*qDsin )
wqDsin2sum = np.sum( sweights_B0*np.square(qDsin) )
w2qDsin2sum = np.sum( np.square(sweights_B0*qDsin) )

Sasymmetry    = wqDsinsum / wqDsin2sum
Sasymmetryerr = np.sqrt(w2qDsin2sum) / wqDsin2sum
print("S = %5.3f +/- %5.3f" %(Sasymmetry,Sasymmetryerr))

# the following code extracts S and C simultanously
qDcos = qD * np.cos(decaytime*deltaM)
wqDcossum = np.sum( sweights_B0*qDcos )
wqDcos2sum = np.sum( sweights_B0*np.square(qDcos) )
w2qDcos2sum = np.sum( np.square(sweights_B0*qDcos) )
wqD2cossinsum = np.sum( sweights_B0*qDcos*qDsin )
w2qD2cossinsum = np.sum( np.square(sweights_B0)*qDcos*qDsin )

b = np.array( [ wqDsinsum,wqDcossum] )

A = np.array( [[wqDsin2sum,wqD2cossinsum],[wqD2cossinsum,wqDcos2sum]] )
x = np.linalg.solve(A,b)
print(x/0.803)
S = x[0]
C = x[1]

# the only thing to do is to get the weights squared correction
wqD2cossinsum = np.sum( sweights_B0*qDcos*qDsin )
B = np.array( [[w2qDsin2sum,w2qD2cossinsum],[w2qD2cossinsum,w2qDcos2sum]] )
Ainv = np.linalg.inv(A)
cov = Ainv @ B @ Ainv

print("S = %5.3f +/- %5.3f" %(S,np.sqrt(cov[0,0])))
print("C = %5.3f +/- %5.3f" %(C,np.sqrt(cov[1,1])))


# create a function to superimpose on the asymmetry
x = np.linspace(0,0.015, 1000)
y = S*np.sin(x*deltaM) + C*np.cos(x*deltaM)
plt.errorbar(x=avtime, y=asymmetry, xerr=xerrors, yerr=asymmetryerr,fmt='o')
plt.plot(x,y)
plt.show()


